In [1590]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


In [1591]:
int_reactions= list()
i= -1
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()

    flt= float(csvFile.iloc[100,  2])
    
    if v == 1 and csvFile.iloc[100,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[100,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])



['LCTStex']
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'DM_5drib_c', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M', 'EX_14glucan_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_4hoxpacd_e', 'EX_5mtr_e', 'EX_cys__D_e', 'EX_acgal_e', 'EX_inost_e', 'EX_k_e', 'EX_kdo2lipid4_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_lipa_e', 'EX_lipa_cold_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_met__L_e', 'EX_metsox_R__L_e', 'EX_metsox_S__L_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_e', 'EX_nmn_e', 'EX_dms_e', 'EX_novbcn_e', 'EX_o16

In [1592]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    i3= 101

    while i3>100 and i3<len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4<100:
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [1593]:
"NTD6" in diff_indecr_both_positive_and_negative

False

In [1594]:
i= 0
indecr_and_stable= list()
diff_indecr= list()
flat= list()
flat2= list()
flat3= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])


    if len(d)> 30 and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable.append(r_id)

        elif count> 0.95*len(d):
            flat.append(r_id)
        
        else:
            diff_indecr.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable.append(r_id)

        elif count> 1 and len(d) < 3:
            flat.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3.append(r_id)
        
        else:
            diff_indecr.append(r_id)
    
    i= i+1


flat
diff_indecr
indecr_and_stable



['FRD3', 'O2tex', 'SPODM']

In [1595]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
flat2_below_zero= list()
flat3_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])


    if len(d)> 30 and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2_below_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1


flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero


['EX_csn_e',
 'EX_glcur_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_h2_e',
 'EX_acgam1p_e',
 'EX_man_e',
 'EX_manglyc_e',
 'EX_o2_e',
 'EX_ppa_e',
 'EX_ppal_e',
 'EX_ppt_e',
 'EX_psclys_e',
 'EX_r5p_e',
 'EX_fald_e',
 'EX_uacgam_e',
 'EX_udpg_e',
 'EX_udpgal_e',
 'EX_fru_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_g1p_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'ACOLIPAabctex',
 'EX_galctn__D_e',
 'EX_galctn__L_e',
 'EX_gam_e',
 'EX_gam6p_e',
 'CHLabcpp',
 'CLIPAabctex',
 'COLIPAabctex',
 'CU1abcpp',
 'DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'ECA4COLIPAabctex',
 'DSBAO2',
 'ENLIPAabctex',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GLYBabcpp',
 'HYD2pp',
 'HYD3pp',
 'HG2abcpp',
 'LIPAabctex',
 'NO3R1bpp',
 'NO3R1pp',
 'NO3R2bpp',
 'NTRIR2x',
 'NTRIR3pp',
 'NTRIR4pp',
 'QMO2',
 'TMAOR1',
 'TMAOR1pp',
 'TMAOR2',
 'TMAOR2pp',
 'XYLabcpp',
 'PPPGO',
 'DKMPPD']

In [1596]:
flat3_below_zero

flat_below_zero.append("EX_minohp_e")
flat_below_zero.append("CU1Opp")
len(flat_below_zero)

58

In [1597]:
i= 0
indecr_and_stable_both_positive_and_negative= list()
diff_indecr_both_positive_and_negative= list()
flat_both_positive_and_negative= list()
flat2_both_positive_and_negative= list()
flat3_both_positive_and_negative= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])


    if len(d)> 30 and r_id in l3 and r_id in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_both_positive_and_negative.append(r_id)

        elif count> 0.95*len(d):
            flat_both_positive_and_negative.append(r_id)
        
        else:
            diff_indecr_both_positive_and_negative.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_both_positive_and_negative.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_both_positive_and_negative.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2_both_positive_and_negative.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3_both_positive_and_negative.append(r_id)
        
        else:
            diff_indecr_both_positive_and_negative.append(r_id)
    
    i= i+1

flat_both_positive_and_negative
diff_indecr_both_positive_and_negative
indecr_and_stable_both_positive_and_negative


['EX_glc__D_e',
 'EX_glcn_e',
 'EX_glyclt_e',
 'EX_h_e',
 'EX_h2o_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'EX_ac_e',
 'EX_agm_e',
 'EX_lys__L_e',
 'EX_alaala_e',
 'EX_arg__L_e',
 'EX_orn_e',
 'EX_fe2_e',
 'EX_fe3_e',
 'ACACT5r',
 'ACACT6r',
 'ACACT7r',
 'ACACT8r_copy1',
 'ACKr',
 'ALAALAD',
 'ADA',
 'ADD',
 'ALATA_D2',
 'ALATA_L2',
 'ALAabcpp',
 'ALDD2x',
 'ADNK1',
 'ADNUC',
 'ALDD2y',
 'ADPT',
 'AMPN',
 'ASPabcpp',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'ASPt2pp_copy2',
 'ATPM',
 'CBPS',
 'CD2abcpp',
 'CMtpp',
 'COBALT2abcpp',
 'CRNDabcpp',
 'CRNDt2rpp',
 'CRNabcpp',
 'DADA',
 'CTBTabcpp',
 'CTECOAI7_copy1',
 'CTECOAI8_copy1',
 'CYSabc2pp',
 'CYTBD2pp',
 'CYTBDpp',
 'DHAPT',
 'ARBabcpp',
 'DAAD',
 'ARBt3ipp',
 'DHORD5',
 'DOXRBCNtpp',
 'ARGDCpp',
 'EAR120y',
 'ARGabcpp',
 'EAR140y',
 'EAR160y',
 'F6PP',
 'EAR161y',
 'EAR180y',
 'FA120ACPHi',
 'FA140ACPHi',
 'FA141ACPHi',
 'FA160ACPHi',
 'FA161ACPHi',
 'EAR181y',
 'FADRx2',
 'FDH4pp',
 'ASNN',
 'FDH5pp',
 'FACOAL60t2pp',
 'ASNNpp',
 'ASNS1',
 'FEROpp

In [1598]:
i2= 0
r_id = indecr_and_stable[0]
mon_incr_and_stable_only_above_zero= list()
mon_decr_and_stable_only_above_zero= list()

for r_id in indecr_and_stable:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_only_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_only_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable)
set2= set(mon_decr_and_stable_only_above_zero)
set3= set(mon_incr_and_stable_only_above_zero)
inANDdecr_and_stable= list(set1-set2-set3)
inANDdecr_and_stable


['O2tex']

In [1599]:
i2= 0
r_id = indecr_and_stable_below_zero[0]
mon_incr_and_stable_only_below_zero= list()
mon_decr_and_stable_only_below_zero= list()

for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_only_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_only_below_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_only_below_zero)
set3= set(mon_incr_and_stable_only_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


['EX_manglyc_e',
 'NTRIR2x',
 'EX_galctn__D_e',
 'EX_ppal_e',
 'EX_g1p_e',
 'EX_galctn__L_e',
 'EX_h2_e',
 'NTRIR4pp',
 'EX_man_e',
 'EX_udpg_e',
 'EX_o2_e',
 'EX_uacgam_e',
 'NO3R1bpp',
 'EX_gam_e',
 'PPPGO',
 'EX_udpgal_e',
 'EX_ppa_e',
 'NTRIR3pp',
 'EX_fru_e']

In [1600]:
csvFile = pd.read_csv('biomass_per_flux_for_reaction_EX_h2_e_lactose.csv')
l= list()
i=0

while i < 201:
    inf= csvFile.iloc[i,  2]
    flt= float(csvFile.iloc[i,  2])
    
    if csvFile.iloc[i,  2] == flt:
        l.append(csvFile.iloc[i,  2])
    i= i+1

csvFile2= pd.DataFrame (l, columns = ['Biomass'])
csvFile2.Biomass = csvFile2.Biomass.round(2)
l= list(csvFile2.Biomass)

my_series = csvFile2['Biomass'].squeeze()

n2=0

if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and l[n2+4] == max(l):
    print("no")

else:
    print("yes")

    
n2= len(l)-1

if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and l[n2-4] < max(l) or l[n2-4] == max(l):
    print("no")

else:
    print("yes")

yes
no


In [1601]:
w= list()
w2= list()
i2=0

while i2 < len(indecr_and_stable_below_zero):
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w2.append(r_id)

    
    i2= i2+1

w

['EX_galctn__D_e', 'EX_galctn__L_e', 'NTRIR4pp', 'PPPGO']

In [1602]:
w2

[]

In [1603]:
inANDdecr_and_stable_below_zero.remove("NTRIR4pp")
inANDdecr_and_stable_below_zero.remove("PPPGO")
inANDdecr_and_stable_below_zero.remove("EX_galctn__D_e")
inANDdecr_and_stable_below_zero.remove("EX_galctn__L_e")
inANDdecr_and_stable_below_zero


['EX_manglyc_e',
 'NTRIR2x',
 'EX_ppal_e',
 'EX_g1p_e',
 'EX_h2_e',
 'EX_man_e',
 'EX_udpg_e',
 'EX_o2_e',
 'EX_uacgam_e',
 'NO3R1bpp',
 'EX_gam_e',
 'EX_udpgal_e',
 'EX_ppa_e',
 'NTRIR3pp',
 'EX_fru_e']

In [1604]:
mon_decr_and_stable_only_below_zero.append("NTRIR4pp")
mon_decr_and_stable_only_below_zero.append("PPPGO")
mon_decr_and_stable_only_below_zero.append("EX_galctn__D_e")
mon_incr_and_stable_only_below_zero.append("EX_galctn__L_e")
mon_decr_and_stable_only_below_zero



['EX_csn_e',
 'EX_glcur_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_acgam1p_e',
 'EX_ppt_e',
 'EX_psclys_e',
 'EX_r5p_e',
 'EX_fald_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'ACOLIPAabctex',
 'EX_gam6p_e',
 'CHLabcpp',
 'CLIPAabctex',
 'COLIPAabctex',
 'CU1abcpp',
 'ECA4COLIPAabctex',
 'ENLIPAabctex',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GLYBabcpp',
 'HG2abcpp',
 'LIPAabctex',
 'NO3R2bpp',
 'XYLabcpp',
 'DKMPPD',
 'NTRIR4pp',
 'PPPGO',
 'EX_galctn__D_e']

In [1605]:
w3= list()
w4= list()
i2=0

while i2 < len(indecr_and_stable):
    
    i=0
    r_id = indecr_and_stable[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w3.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w4.append(r_id)

    
    i2= i2+1

w3

[]

In [1606]:
i2= 0
r_id = indecr_and_stable_both_positive_and_negative[0]
mon_incr_and_stable_both_positive_and_negative= list()
mon_decr_and_stable_both_positive_and_negative= list()

for r_id in indecr_and_stable_both_positive_and_negative:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_both_positive_and_negative.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_both_positive_and_negative)
set2= set(mon_decr_and_stable_both_positive_and_negative)
set3= set(mon_incr_and_stable_both_positive_and_negative)
inANDdecr_and_stable_both_positive_and_negative= list(set1-set2-set3)
inANDdecr_and_stable_both_positive_and_negative


['NTD9',
 'ARGDCpp',
 'EX_fe3_e',
 'GLUSy',
 'Htex',
 'NI2uabcpp',
 'FDH5pp',
 'PYRDC',
 'GLYCK2',
 'GTPDPDP',
 'H2Otex',
 'EX_agm_e',
 'NADH18pp',
 'EX_ac_e',
 'HEX7',
 'ALATA_L2',
 'ZNabcpp',
 'PIuabcpp',
 'NADH16pp',
 'HXCT',
 'ASNN',
 'FRD2',
 'SULabcpp',
 'GLYCTO4',
 'NTD8',
 'INSK',
 'PTAr',
 'FDH4pp',
 'URIK2',
 'PRPPS',
 'DHAPT',
 'EX_h2o_e',
 'EX_h_e',
 'GART',
 'DHORD5',
 'PFK_3',
 'GLYCTO2',
 'Kabcpp',
 'PPA',
 'GLYCTO3',
 'NADH17pp',
 'GSNK',
 'ADNK1',
 'CBPS',
 'FEROpp',
 'PDH',
 'EX_glc__D_e',
 'EX_lys__L_e',
 'SUCDi',
 'MNt2pp',
 'ACKr',
 'ASNS1']

In [1607]:
"PTAr" in inANDdecr_and_stable_both_positive_and_negative

True

In [1608]:
w5= list()
w6= list()
i2=0

while i2 < len(inANDdecr_and_stable_both_positive_and_negative):
    
    i=0
    r_id = inANDdecr_and_stable_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    i2= i2+1
    
w5

['NTD9',
 'ARGDCpp',
 'GLUSy',
 'NI2uabcpp',
 'PYRDC',
 'GLYCK2',
 'GTPDPDP',
 'HEX7',
 'ALATA_L2',
 'ZNabcpp',
 'PIuabcpp',
 'HXCT',
 'SULabcpp',
 'NTD8',
 'INSK',
 'FDH4pp',
 'URIK2',
 'PRPPS',
 'DHAPT',
 'EX_h_e',
 'GART',
 'PFK_3',
 'GLYCTO2',
 'Kabcpp',
 'PPA',
 'GSNK',
 'ADNK1',
 'CBPS',
 'FEROpp',
 'PDH',
 'MNt2pp',
 'ACKr',
 'ASNS1']

In [1609]:
w6

['EX_fe3_e', 'Htex', 'PTAr', 'DHORD5', 'EX_lys__L_e']

In [1610]:
inANDdecr_and_stable_both_positive_and_negative.remove("PTAr")
inANDdecr_and_stable_both_positive_and_negative.remove("Htex")
inANDdecr_and_stable_both_positive_and_negative.remove("EX_fe3_e")
inANDdecr_and_stable_both_positive_and_negative.remove("DHORD5")
inANDdecr_and_stable_both_positive_and_negative.remove("EX_lys__L_e")

inANDdecr_and_stable_both_positive_and_negative.remove("ASNS1")
inANDdecr_and_stable_both_positive_and_negative.remove("ACKr")
inANDdecr_and_stable_both_positive_and_negative.remove("MNt2pp")
inANDdecr_and_stable_both_positive_and_negative.remove("PDH")
inANDdecr_and_stable_both_positive_and_negative.remove("FEROpp")
inANDdecr_and_stable_both_positive_and_negative.remove("CBPS")
inANDdecr_and_stable_both_positive_and_negative.remove("ADNK1")
inANDdecr_and_stable_both_positive_and_negative.remove("GSNK")
inANDdecr_and_stable_both_positive_and_negative.remove("PPA")
inANDdecr_and_stable_both_positive_and_negative.remove("Kabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("GLYCTO2")
inANDdecr_and_stable_both_positive_and_negative.remove("PFK_3")
inANDdecr_and_stable_both_positive_and_negative.remove("GART")
inANDdecr_and_stable_both_positive_and_negative.remove("EX_h_e")
inANDdecr_and_stable_both_positive_and_negative.remove("DHAPT")
inANDdecr_and_stable_both_positive_and_negative.remove("PRPPS")
inANDdecr_and_stable_both_positive_and_negative.remove("URIK2")
inANDdecr_and_stable_both_positive_and_negative.remove("FDH4pp")
inANDdecr_and_stable_both_positive_and_negative.remove("INSK")
inANDdecr_and_stable_both_positive_and_negative.remove("NTD8")
inANDdecr_and_stable_both_positive_and_negative.remove("SULabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("HXCT")
inANDdecr_and_stable_both_positive_and_negative.remove("PIuabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("ZNabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("ALATA_L2")
inANDdecr_and_stable_both_positive_and_negative.remove("HEX7")
inANDdecr_and_stable_both_positive_and_negative.remove("GTPDPDP")
inANDdecr_and_stable_both_positive_and_negative.remove("GLYCK2")
inANDdecr_and_stable_both_positive_and_negative.remove("PYRDC")
inANDdecr_and_stable_both_positive_and_negative.remove("NI2uabcpp")
inANDdecr_and_stable_both_positive_and_negative.remove("GLUSy")
inANDdecr_and_stable_both_positive_and_negative.remove("ARGDCpp")
inANDdecr_and_stable_both_positive_and_negative.remove("NTD9")


In [1611]:
increasing_below_zero_and_flat_above_zero= ["EX_fe3_e", "PTAr", "Htex"]
increasing_and_stable_below_zero_and_decreasing_above_zero= ["DHORD5", "EX_lys__L_e", "GLYCK2"]
flat_below_zero_and_decreasing_above_zero= ["ASNS1", "ACKr", "PDH", "ADNK1", "GSNK", "GLYCTO2", "PFK_3", "GART", "EX_h_e", "DHAPT", "PRPPS", "URIK2", "FDH4pp", "INSK", "NTD8", "HXCT", "ALATA_L2", "HEX7", "GTPDPDP", "PYRDC", "GLUSy", "NTD9"]
decreasing_and_stable_below_zero_and_decreasing_above_zero= ["MNt2pp", "CBPS", "PPA", "Kabcpp", "SULabcpp", "PIuabcpp", "ZNabcpp", "NI2uabcpp", "ARGDCpp"]
flat_below_zero_and_decreasing_and_stable_above_zero= ["FEROpp"]


In [1612]:
i=0
A= list()
b= list()
c= list()
d= list()
e= list()


while i < len(mon_decr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_decr_and_stable_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        A.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 3:
            decreasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        else:
            flat_below_zero_and_decreasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
    
    if boolean == False and boolean2 == False:
        b.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == True:
        c.append(mon_decr_and_stable_both_positive_and_negative[i])

    if boolean == True:
        d.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean2 == True:
        e.append(mon_decr_and_stable_both_positive_and_negative[i])
        
    i=i+1

decreasing_and_stable_below_zero_and_decreasing_above_zero

['MNt2pp',
 'CBPS',
 'PPA',
 'Kabcpp',
 'SULabcpp',
 'PIuabcpp',
 'ZNabcpp',
 'NI2uabcpp',
 'ARGDCpp',
 'EX_glcn_e',
 'EX_glyclt_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'EX_alaala_e',
 'EX_arg__L_e',
 'EX_orn_e',
 'ALAALAD',
 'ALAabcpp',
 'ALDD2x',
 'ALDD2y',
 'ASPabcpp',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'ATPM',
 'CD2abcpp',
 'CMtpp',
 'COBALT2abcpp',
 'CRNDabcpp',
 'CRNabcpp',
 'CTBTabcpp',
 'CYSabc2pp',
 'CYTBD2pp',
 'CYTBDpp',
 'ARBabcpp',
 'ARBt3ipp',
 'DOXRBCNtpp',
 'ARGabcpp',
 'FE2abcpp',
 'FE2t2pp',
 'ASNabcpp',
 'FUSAtpp',
 'G3PD5',
 'G3PD6',
 'G3PD7',
 'GLCabcpp',
 'GALabcpp',
 'GLUabcpp',
 'GUAt2pp',
 'GLYC3Pabcpp',
 'ILEabcpp',
 'HISabcpp',
 'LCTSt3ipp',
 'LDH_D2',
 'LEUabcpp',
 'LYSabcpp',
 'MALt2_3pp',
 'MINCYCtpp',
 'MDH2',
 'MDH3',
 'MG2tpp',
 'MG2uabcpp',
 'NADH10',
 'NADH5',
 'NADH9',
 'NADPHQR2',
 'NADPHQR3',
 'NADPHQR4',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'NI2abcpp',
 'GTHPi',
 'NTP1',
 'NTP10',
 'NTP3',
 'NTP5',
 'NTPP1',
 'NTPP2',
 'NTPP3',
 'NTPP4',
 'NTPP5',
 'NT

In [1613]:
e

flat_below_zero_and_decreasing_above_zero.append("HEX1")
flat_below_zero_and_decreasing_above_zero.append("PFK")


In [1614]:
W= list(decreasing_and_stable_below_zero_and_decreasing_above_zero)+list(flat_below_zero_and_decreasing_above_zero)

In [1615]:
union = pd.Series(np.union1d(A, W))
intersect = pd.Series(np.intersect1d(A, W))
BBB= union[~union.isin(intersect)]
BBB


1           ACKr
4          ADNK1
9       ALATA_L2
16       ARGDCpp
19         ASNS1
27          CBPS
42         DHAPT
56        EX_h_e
67        FDH4pp
80          GART
84         GLUSy
90       GLYCTO2
92          GSNK
94       GTPDPDP
97          HEX1
98          HEX7
101         HXCT
106         INSK
107       Kabcpp
130       MNt2pp
141    NI2uabcpp
144         NTD8
145         NTD9
162          PDH
163          PFK
164        PFK_3
165     PIuabcpp
167          PPA
172        PRPPS
174        PYRDC
182     SULabcpp
193        URIK2
199      ZNabcpp
dtype: object

In [1616]:
z=0
f= list()
g= list()
h= list()
i= list()
j= list()
flat_below_zero_and_increasing_above_zero= list()

while z < len(mon_incr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(mon_incr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        f.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 3:
            increasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        else:
            flat_below_zero_and_increasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        g.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        h.append(mon_incr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        i.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        j.append(mon_incr_and_stable_both_positive_and_negative[z])
        
    z=z+1

increasing_and_stable_below_zero_and_decreasing_above_zero


['DHORD5', 'EX_lys__L_e', 'GLYCK2']

In [1617]:
h

['ACACT5r',
 'ACACT6r',
 'ACACT7r',
 'CRNDt2rpp',
 'FE2tex',
 'HACD5',
 'HACD6',
 'HACD7',
 'HACD8',
 'DURAD']

In [1618]:
increasing_below_zero_and_flat_above_zero.append("ACACT5r")
increasing_below_zero_and_flat_above_zero.append("ACACT6r")
increasing_below_zero_and_flat_above_zero.append("ACACT7r")
increasing_below_zero_and_flat_above_zero.append("CRNDt2rpp")
increasing_below_zero_and_flat_above_zero.append("FE2tex")
increasing_below_zero_and_flat_above_zero.append("HACD5")
increasing_below_zero_and_flat_above_zero.append("HACD6")
increasing_below_zero_and_flat_above_zero.append("HACD7")
increasing_below_zero_and_flat_above_zero.append("HACD8")
increasing_below_zero_and_flat_above_zero.append("DURAD")

increasing_below_zero_and_flat_above_zero


['EX_fe3_e',
 'PTAr',
 'Htex',
 'ACACT5r',
 'ACACT6r',
 'ACACT7r',
 'CRNDt2rpp',
 'FE2tex',
 'HACD5',
 'HACD6',
 'HACD7',
 'HACD8',
 'DURAD']

In [1619]:
z=0
k= list()
L= list()
m= list()
o= list()
p= list()
increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero= list()
flat_below_zero_and_increasingANDdecreasing_above_zero= list()

while z < len(inANDdecr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(inANDdecr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        k.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 3:
            increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        else:
            flat_below_zero_and_increasingANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        L.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        m.append(inANDdecr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        o.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        p.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
    z=z+1

increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero

['FDH5pp',
 'H2Otex',
 'EX_agm_e',
 'EX_ac_e',
 'ASNN',
 'GLYCTO4',
 'GLYCTO3',
 'EX_glc__D_e']

In [1620]:
m

['NADH18pp', 'FRD2', 'EX_h2o_e', 'NADH17pp', 'SUCDi']

In [1621]:
p

increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero= ["NADH16pp"]
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero


['NADH16pp']

In [1622]:
increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero= ["H2Otex", "EX_agm_e", "EX_ac_e", "EX_glc__D_e"]
increasing_and_stable_below_zero_and_decreasing_above_zero.append("FDH5pp")
increasing_and_stable_below_zero_and_decreasing_above_zero.append("GLYCTO4")
increasing_and_stable_below_zero_and_decreasing_above_zero.append("GLYCTO3")
increasing_below_zero_and_increasingANDdecreasing_and_stable_above_zero= ["EX_h2o_e"]
increasingANDdecreasing_below_zero_and_decreasing_above_zero= ["ASNN"]
increasing_below_zero_and_decreasing_and_stable_above_zero= ["NADH18pp", "FRD2", "NADH17pp", "SUCDi"]


In [1623]:
i2= 0
r_id = diff_indecr[0]
incr_above_zero= list()
decr_above_zero= list()
one_to_three_data_point= list()

for r_id in diff_indecr:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point.append(r_id)

    
    i2= i2 + 1

decr_above_zero


['DM_oxam_c',
 'EX_h2s_e',
 '2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_urea_e',
 '2MAHMP',
 '3HAD100',
 '3HAD120',
 '3HAD121',
 '3HAD140',
 '3HAD141',
 '3HAD160',
 '3HAD161',
 'AACPS8',
 '3HAD180',
 '3HAD181',
 'AACPS9',
 'AACTOOR',
 'ABTA',
 'ABUTD',
 '3OAR100',
 '3OAR120',
 '3OAR121',
 '3OAR141',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAS100',
 '3OAS120',
 '3OAS121',
 '3OAS141',
 '3OAS160',
 '3OAS161',
 '3OAS180',
 '3OAS181',
 'ACGAMK',
 '3PEPTabcpp',
 'ACM6PH',
 'ACOAD4f',
 'ACOAD5f',
 'ACOAD6f',
 'ACOAD7f',
 'ACOAD8f',
 'ACPPAT120',
 'ACPPAT140',
 'ACPPAT141',
 'AGPAT181',
 'ACPPAT180',
 'ACPPAT181',
 'ALAGLUE',
 'ADNCYC',
 'ADPRDP',
 'ALLTAMH',
 'ALLTN',
 '

In [1624]:
incr_above_zero

[]

In [1625]:
set1= set(diff_indecr)
set2= set(decr_above_zero)
set3= set(incr_above_zero)
set4= set(one_to_three_data_point)
inANDdecr_above_zero= list(set1-set2-set3-set4)
inANDdecr_above_zero


['PRAGSr',
 'AGPAT161',
 'G5SADs',
 'ADSS',
 'PSP_L',
 '3OAS140',
 'AGPAT160',
 'PE161abcpp',
 'DTMPK',
 'P5CR',
 'MCOATA',
 'UAAGDS',
 'CS',
 'OCBT',
 'COBALT2tpp',
 'HSK',
 'ACODA',
 'UAGCVT',
 'BPNT',
 'PRFGS',
 'HCO3E',
 'PSERT',
 'NDPK4',
 'PSD160',
 'ADSL1r',
 'PSSA161',
 'ACGS',
 'G6PDH2r',
 'ALAR',
 'CU2tpp',
 'UGMDDS',
 'NH4tex',
 'AIRC2',
 'GLNS',
 'UAMAS',
 'SADT2',
 'ICDHyr',
 'CTPS2',
 'O2tpp',
 '3OAR140',
 'SULR',
 'PE160abcpp',
 'PSSA160',
 'NH4tpp',
 'ASPK',
 'UAGDP',
 'PAPPT3',
 'ACPPAT160',
 'ARGSS',
 'GLUPRT',
 'UAMAGS',
 'THRD_L',
 'SO4tex',
 'NDPK2',
 'PMPK',
 'GK1',
 'ADSK',
 'AICART',
 'SERAT',
 'PGCD',
 'OMPDC',
 'PRASCSi',
 'ADSL2r',
 'GMPS2',
 'ACPPAT161',
 'UAGPT3',
 'ACONTb',
 'ARGSL',
 'ACCOAC',
 'DASYN160',
 'UMPK',
 'ACONTa',
 'NADS1',
 'APG3PAT161',
 'PRAIS',
 'TMPK',
 'UDCPDP',
 'ACGK',
 'NNATr',
 'ASPCT',
 'UAPGR',
 'DASYN161',
 'PSD161',
 'GF6PTA',
 'ALAALAr',
 'THRS',
 'CYSS',
 'APG3PAT160',
 'G1PACT',
 'NADK']

In [1626]:
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_above_zero):
    
    i=0
    r_id = inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w7

['PRAGSr',
 'AGPAT161',
 'G5SADs',
 'ADSS',
 'PSP_L',
 '3OAS140',
 'AGPAT160',
 'PE161abcpp',
 'DTMPK',
 'P5CR',
 'MCOATA',
 'UAAGDS',
 'CS',
 'OCBT',
 'COBALT2tpp',
 'HSK',
 'ACODA',
 'UAGCVT',
 'BPNT',
 'PRFGS',
 'HCO3E',
 'PSERT',
 'NDPK4',
 'PSD160',
 'ADSL1r',
 'PSSA161',
 'ACGS',
 'G6PDH2r',
 'ALAR',
 'CU2tpp',
 'UGMDDS',
 'NH4tex',
 'AIRC2',
 'GLNS',
 'UAMAS',
 'SADT2',
 'ICDHyr',
 'CTPS2',
 '3OAR140',
 'SULR',
 'PE160abcpp',
 'PSSA160',
 'NH4tpp',
 'ASPK',
 'UAGDP',
 'PAPPT3',
 'ACPPAT160',
 'ARGSS',
 'GLUPRT',
 'UAMAGS',
 'THRD_L',
 'SO4tex',
 'NDPK2',
 'PMPK',
 'GK1',
 'ADSK',
 'AICART',
 'SERAT',
 'PGCD',
 'OMPDC',
 'PRASCSi',
 'ADSL2r',
 'GMPS2',
 'ACPPAT161',
 'UAGPT3',
 'ACONTb',
 'ARGSL',
 'ACCOAC',
 'DASYN160',
 'UMPK',
 'ACONTa',
 'NADS1',
 'APG3PAT161',
 'PRAIS',
 'TMPK',
 'UDCPDP',
 'ACGK',
 'NNATr',
 'ASPCT',
 'UAPGR',
 'DASYN161',
 'PSD161',
 'GF6PTA',
 'ALAALAr',
 'THRS',
 'CYSS',
 'APG3PAT160',
 'G1PACT',
 'NADK']

In [1627]:
set1= set(inANDdecr_above_zero)
set2= set(w7)
set3= set(w8)
truly_inANDdecr_above_zero= list(set1-set2-set3)
truly_inANDdecr_above_zero


['O2tpp']

In [1628]:
ww= w7 + w8
ww

ww = list(set(ww))

decr_above_zero= decr_above_zero + ww
decr_above_zero

['DM_oxam_c',
 'EX_h2s_e',
 '2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_urea_e',
 '2MAHMP',
 '3HAD100',
 '3HAD120',
 '3HAD121',
 '3HAD140',
 '3HAD141',
 '3HAD160',
 '3HAD161',
 'AACPS8',
 '3HAD180',
 '3HAD181',
 'AACPS9',
 'AACTOOR',
 'ABTA',
 'ABUTD',
 '3OAR100',
 '3OAR120',
 '3OAR121',
 '3OAR141',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAS100',
 '3OAS120',
 '3OAS121',
 '3OAS141',
 '3OAS160',
 '3OAS161',
 '3OAS180',
 '3OAS181',
 'ACGAMK',
 '3PEPTabcpp',
 'ACM6PH',
 'ACOAD4f',
 'ACOAD5f',
 'ACOAD6f',
 'ACOAD7f',
 'ACOAD8f',
 'ACPPAT120',
 'ACPPAT140',
 'ACPPAT141',
 'AGPAT181',
 'ACPPAT180',
 'ACPPAT181',
 'ALAGLUE',
 'ADNCYC',
 'ADPRDP',
 'ALLTAMH',
 'ALLTN',
 '

In [1629]:
i2= 0
r_id = diff_indecr_below_zero[0]
incr_below_zero= list()
decr_below_zero= list()
one_to_three_data_point2= list()

for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_below_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point2.append(r_id)

    
    i2= i2 + 1

decr_below_zero


['EX_cmp_e',
 'EX_glcur1p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_23camp_e',
 'EX_23cgmp_e',
 'EX_23cump_e',
 'EX_2ddglcn_e',
 'EX_3amp_e',
 'EX_3cmp_e',
 'EX_3gmp_e',
 'EX_3ump_e',
 'EX_cytd_e',
 'EX_dad_2_e',
 'EX_imp_e',
 'EX_acac_e',
 'EX_damp_e',
 'EX_ins_e',
 'EX_adn_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_mal__D_e',
 'EX_dgmp_e',
 'EX_man6p_e',
 'EX_amp_e',
 'EX_dgsn_e',
 'EX_dimp_e',
 'EX_din_e',
 'EX_dtmp_e',
 'EX_dump_e',
 'EX_duri_e',
 'EX_pser__L_e',
 'EX_rmn_e',
 'EX_ser__D_e',
 'EX_f6p_e',
 'EX_thrp_e',
 'EX_thymd_e',
 'EX_trp__L_e',
 'EX_ump_e',
 'EX_uri_e',
 'EX_g3pc_e',
 'EX_g3pi_e',
 'EX_g3ps_e',
 'EX_g6p_e']

In [1630]:
incr_below_zero

['EX_cpgn_e',
 'EX_isetac_e',
 'EX_arbtn_fe3_e',
 'EX_mso3_e',
 'EX_no_e',
 'EX_no2_e',
 'EX_dmso_e',
 'EX_no3_e',
 'EX_orot_e',
 'EX_skm_e',
 'EX_sulfac_e',
 'EX_taur_e',
 'EX_tmao_e',
 'EX_fe3dhbzs_e',
 '2DGULRGx',
 'EX_fe3hox_e',
 'EX_fecrm_e',
 'EX_feenter_e',
 'EX_feoxam_e',
 'ACALDtex',
 'ACALDtpp',
 'ACtex',
 'ADK4',
 'ALAtex',
 'ALCD19',
 'ALCD2x',
 'ALDD3y',
 'ALLTNt2rpp',
 'ALLTNtex',
 'ALPATE160pp',
 'ALPATG160pp',
 'ASPtex',
 'ATHRDHr',
 'BALAt2pp',
 'CHLt2pp',
 'BETALDHy',
 'BSORx',
 'CSNt2pp',
 'CTBTt2rpp',
 'CUt3',
 'DHAtex',
 'DHAtpp',
 'D_LACt2pp',
 'D_LACtex',
 'DOGULNR',
 'DSERDHr',
 'DSERt2pp',
 'ECOAH4',
 'ECOAH5',
 'ECOAH6',
 'ECOAH7',
 'ECOAH8',
 'ETOHtex',
 'ETOHtrpp',
 'FESD2s',
 'FORtex',
 'ASNt2rpp',
 'FRUpts2pp',
 'ASNtex',
 'FUMt2_2pp',
 'G2PPpp',
 'ASPO6',
 'GLNabcpp',
 'GLYCAtex',
 'GLUNpp',
 'GLYALDtex',
 'GLYALDtpp',
 'GLYBt2pp',
 'GUAtpp',
 'GLYCAt2rpp',
 'H2Stex',
 'GLYCLTtex',
 'HEXt2rpp',
 'HG2t3pp',
 'ILEt2rpp',
 'GLYCtex',
 'HISt2rpp',
 'L_LACt2rp

In [1631]:
set1= set(diff_indecr_below_zero)
set2= set(decr_below_zero)
set3= set(incr_below_zero)
set4= set(one_to_three_data_point2)
inANDdecr_below_zero= list(set1-set2-set3-set4)
inANDdecr_below_zero


['EX_LalaDgluMdap_e',
 'EX_acmana_e',
 'ASAD',
 'EX_lcts_e',
 'ORPT',
 'EX_xyl__D_e',
 'EX_sbt__D_e',
 'EX_malt_e',
 'EX_ethso3_e',
 'IMPC',
 'EX_so4_e',
 'EX_ascb__L_e',
 'EX_lyx__L_e',
 'RPI',
 'EX_3hcinnm_e',
 'EX_fum_e',
 'EX_ttdcea_e',
 'EX_galct__D_e',
 'EX_fe3dcit_e',
 'EX_26dap__M_e',
 'EX_udpglcur_e',
 'EX_pppn_e',
 'EX_acgam_e',
 'EX_arab__L_e',
 'EX_LalaDgluMdapDala_e',
 'EX_xylu__L_e',
 'EX_but_e',
 'DHORTS',
 'EX_gthrd_e',
 'EX_23dappa_e',
 'EX_butso3_e',
 'AIRC3',
 'EX_LalaLglu_e',
 'OMPHHX3',
 'EX_chtbs_e',
 'GLUR',
 'EX_melib_e',
 'OPHHX3',
 'EX_acmum_e',
 'EX_hdca_e',
 'EX_tartr__D_e',
 'EX_ttdca_e',
 'OMMBLHX3',
 'EX_nh4_e',
 'EX_glcr_e',
 'EX_sucr_e',
 'EX_rib__D_e',
 'EX_ocdcea_e',
 'ACOTA',
 'AGPR',
 'G3PD2',
 'EX_tre_e',
 'EX_3hpppn_e',
 'EX_mnl_e',
 'EX_galt_e',
 'EX_dca_e',
 'EX_anhgm_e',
 'HSDy',
 'EX_hdcea_e',
 'EX_tartr__L_e',
 'EX_23ccmp_e',
 'EX_acnam_e',
 'EX_LalaDglu_e',
 'EX_progly_e',
 'PGAMT',
 'EX_udpacgal_e',
 'EX_octa_e',
 'EX_ocdca_e',
 'ASPTA',
 '

In [1632]:
w9= list()
w10= list()
i2=0

while i2 < len(inANDdecr_below_zero):
    
    i=0
    r_id = inANDdecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w9.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w10.append(r_id)
    
    i2= i2+1
    
w9

['ORPT',
 'IMPC',
 'EX_so4_e',
 'EX_LalaDgluMdapDala_e',
 'DHORTS',
 'AIRC3',
 'GLUR']

In [1633]:
w10

['ASAD',
 'ORPT',
 'IMPC',
 'EX_so4_e',
 'RPI',
 'EX_26dap__M_e',
 'DHORTS',
 'AIRC3',
 'GLUR',
 'EX_nh4_e',
 'ACOTA',
 'AGPR',
 'G3PD2',
 'HSDy',
 'PGAMT',
 'ASPTA']

In [1634]:
set1= set(inANDdecr_below_zero)
set2= set(w9)
set3= set(w10)
truly_inANDdecr_below_zero= list(set1-set2-set3)
truly_inANDdecr_below_zero


['EX_LalaDgluMdap_e',
 'EX_acmana_e',
 'EX_lcts_e',
 'EX_xyl__D_e',
 'EX_sbt__D_e',
 'EX_malt_e',
 'EX_ascb__L_e',
 'EX_lyx__L_e',
 'EX_3hcinnm_e',
 'EX_galct__D_e',
 'EX_ttdcea_e',
 'EX_fe3dcit_e',
 'EX_fum_e',
 'EX_udpglcur_e',
 'EX_pppn_e',
 'EX_acgam_e',
 'EX_arab__L_e',
 'EX_xylu__L_e',
 'EX_but_e',
 'EX_gthrd_e',
 'EX_23dappa_e',
 'EX_chtbs_e',
 'EX_butso3_e',
 'EX_LalaLglu_e',
 'EX_melib_e',
 'EX_ethso3_e',
 'OMPHHX3',
 'OPHHX3',
 'EX_acmum_e',
 'EX_hdca_e',
 'EX_tartr__D_e',
 'EX_ttdca_e',
 'OMMBLHX3',
 'EX_sucr_e',
 'EX_glcr_e',
 'EX_rib__D_e',
 'EX_ocdcea_e',
 'EX_tre_e',
 'EX_3hpppn_e',
 'EX_mnl_e',
 'EX_galt_e',
 'EX_dca_e',
 'EX_anhgm_e',
 'EX_LalaDglu_e',
 'EX_hdcea_e',
 'EX_tartr__L_e',
 'EX_23ccmp_e',
 'EX_acnam_e',
 'EX_progly_e',
 'EX_udpacgal_e',
 'EX_octa_e',
 'EX_ocdca_e',
 'EX_cynt_e',
 'EX_galur_e',
 'EX_all__D_e',
 'EX_ddca_e']

In [1635]:
ww= w9 + w10
ww

ww = list(set(ww))

incr_below_zero= incr_below_zero + ww
incr_below_zero


['EX_cpgn_e',
 'EX_isetac_e',
 'EX_arbtn_fe3_e',
 'EX_mso3_e',
 'EX_no_e',
 'EX_no2_e',
 'EX_dmso_e',
 'EX_no3_e',
 'EX_orot_e',
 'EX_skm_e',
 'EX_sulfac_e',
 'EX_taur_e',
 'EX_tmao_e',
 'EX_fe3dhbzs_e',
 '2DGULRGx',
 'EX_fe3hox_e',
 'EX_fecrm_e',
 'EX_feenter_e',
 'EX_feoxam_e',
 'ACALDtex',
 'ACALDtpp',
 'ACtex',
 'ADK4',
 'ALAtex',
 'ALCD19',
 'ALCD2x',
 'ALDD3y',
 'ALLTNt2rpp',
 'ALLTNtex',
 'ALPATE160pp',
 'ALPATG160pp',
 'ASPtex',
 'ATHRDHr',
 'BALAt2pp',
 'CHLt2pp',
 'BETALDHy',
 'BSORx',
 'CSNt2pp',
 'CTBTt2rpp',
 'CUt3',
 'DHAtex',
 'DHAtpp',
 'D_LACt2pp',
 'D_LACtex',
 'DOGULNR',
 'DSERDHr',
 'DSERt2pp',
 'ECOAH4',
 'ECOAH5',
 'ECOAH6',
 'ECOAH7',
 'ECOAH8',
 'ETOHtex',
 'ETOHtrpp',
 'FESD2s',
 'FORtex',
 'ASNt2rpp',
 'FRUpts2pp',
 'ASNtex',
 'FUMt2_2pp',
 'G2PPpp',
 'ASPO6',
 'GLNabcpp',
 'GLYCAtex',
 'GLUNpp',
 'GLYALDtex',
 'GLYALDtpp',
 'GLYBt2pp',
 'GUAtpp',
 'GLYCAt2rpp',
 'H2Stex',
 'GLYCLTtex',
 'HEXt2rpp',
 'HG2t3pp',
 'ILEt2rpp',
 'GLYCtex',
 'HISt2rpp',
 'L_LACt2rp

In [1636]:
i2= 0
r_id = diff_indecr_both_positive_and_negative[0]
incr_both_positive_and_negative= list()
decr_both_positive_and_negative= list()
one_to_three_data_point3= list()

for r_id in diff_indecr_both_positive_and_negative:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_both_positive_and_negative.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point3.append(r_id)

    
    i2= i2 + 1

decr_both_positive_and_negative

['EX_glu__L_e',
 'EX_gly_e',
 'EX_glyald_e',
 'EX_glyc3p_e',
 'EX_5dglcn_e',
 'EX_acald_e',
 'EX_lac__D_e',
 'EX_lac__L_e',
 'EX_akg_e',
 'EX_ala__L_e',
 'EX_mal__L_e',
 'EX_asp__L_e',
 'EX_dha_e',
 'EX_pyr_e',
 'EX_ser__L_e',
 'EX_etha_e',
 'EX_thr__L_e',
 'EX_g3pe_e',
 'EX_g3pg_e']

In [1637]:
incr_both_positive_and_negative

['GLCptspp', 'GALt2pp', 'LACZ', 'PGMT']

In [1638]:
one_to_three_data_point3

['ACOAD1f', 'ACOAD2f', 'ECOAH1', 'ECOAH2']

In [1639]:
decreasing_below_zero_and_decreasing_above_zero= ["EX_glu__L_e", "EX_gly_e", "EX_glyald_e", "EX_glyc3p_e", "EX_5dglcn_e", "EX_acald_e", "EX_lac__D_e", "EX_lac__L_e", "EX_akg_e", "EX_ala__L_e", "EX_mal__L_e", "EX_asp__L_e", "EX_dha_e", "EX_pyr_e", "EX_ser__L_e", "EX_etha_e", "EX_thr__L_e", "EX_g3pe_e", "EX_g3pg_e"]
increasing_below_zero_and_increasing_above_zero= ["GLCptspp", "GALt2pp", "LACZ", "PGMT"]


In [1640]:
set1= set(diff_indecr_both_positive_and_negative)
set2= set(decr_both_positive_and_negative)
set3= set(incr_both_positive_and_negative)
set4= set(one_to_three_data_point3)
inANDdecr_both_positive_and_negative= list(set1-set2-set3-set4)
inANDdecr_both_positive_and_negative


['APH181',
 'PGPP180',
 'EX_quin_e',
 'SUCASPtpp',
 'TKT1',
 'PGPP181',
 'G3PT',
 'EAR121x',
 'EX_ade_e',
 'TRPS3',
 'DCTPD',
 '2AGPEAT180',
 'FECRMR2',
 'EX_pro__L_e',
 'GDPMNH',
 'COBALT2t3pp',
 'CDAPPA140',
 'GLYCDx',
 'PGPP140',
 'RNTR4c2',
 'ATPS4rpp',
 'CYTK2',
 'CYTDH',
 'FMNRx',
 'GAPD',
 'ME2',
 'ARBt2rpp',
 'Kt2pp',
 'MOX',
 'AACPS6',
 'FBA',
 'PROD2',
 'APH180',
 'TMDK1',
 '2AGPGAT180',
 'EX_hxa_e',
 'TPI',
 'ARBTNR3',
 'CYSDS',
 'ORNDC',
 'PGK',
 'FA80ACPHi',
 'RNTR1c',
 'FACOAE180',
 'AGM3PApp',
 'ZN2t3pp',
 'THRt4pp',
 'AACPS5',
 'EX_ala__D_e',
 'EX_gal_e',
 'CPGNR1',
 'ENTERES',
 'FUM',
 'FACOAE141',
 'MLTGY1pp',
 '2AGPGAT181',
 'AKGDH',
 'FACOAE120',
 'AACPS7',
 'AGM3PH',
 'AGM3PA',
 'PGI',
 'AGM4PCPpp',
 'GLUDy',
 '2AGPGAT120',
 'CYTBO3_4pp',
 'MLDEP2pp',
 'H2SO',
 'CDAPPA161',
 'GDPTPDP',
 'PGPP161',
 'PAPSR',
 'PTRCt2pp',
 'AACPS4',
 'MDDCP1pp',
 'LPLIPAL2E160',
 'LPLIPAL2E141',
 'ACOATA',
 'TALA',
 'EX_glyc_e',
 'FACOAE161',
 'PGPP160',
 'SO4t2pp',
 'APH161',
 'OBTF

In [1641]:
w11= list()
w12= list()
i2=0

while i2 < len(inANDdecr_both_positive_and_negative):
    
    i=0
    r_id = inANDdecr_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w11.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w12.append(r_id)
    
    i2= i2+1
    
w11


['EX_quin_e',
 'PROD2',
 'TMDK1',
 'CYSDS',
 'PGK',
 'EX_ala__D_e',
 'MLTGY1pp',
 'MLDEP2pp',
 'MDDCP1pp',
 'OBTFL',
 'G6PDA',
 'NTD1',
 'CYSabcpp',
 'ALAALAabcpp',
 'MLDCP2Bpp',
 'AACPS2',
 'PGL',
 'MDDEP3pp',
 'SPMDabcpp',
 'MLDCP1Bpp',
 'GNK',
 'DURIPP',
 'EX_succ_e',
 'ETHAt2pp',
 'FACOAE80',
 'P5CD']

In [1642]:
union = pd.Series(np.union1d(w11, w12))
set1= set(inANDdecr_both_positive_and_negative)
set2= set(union)
truly_inANDdecr_both_positive_and_negative= list(set1-set2)
truly_inANDdecr_both_positive_and_negative


['APH181',
 'PGPP180',
 'SUCASPtpp',
 'TKT1',
 'PGPP181',
 'G3PT',
 'EX_ade_e',
 'DCTPD',
 '2AGPEAT180',
 'FECRMR2',
 'EX_pro__L_e',
 'GDPMNH',
 'COBALT2t3pp',
 'CDAPPA140',
 'GLYCDx',
 'PGPP140',
 'RNTR4c2',
 'ATPS4rpp',
 'CYTK2',
 'CYTDH',
 'FMNRx',
 'ME2',
 'ARBt2rpp',
 'Kt2pp',
 'MOX',
 'AACPS6',
 'FBA',
 'APH180',
 '2AGPGAT180',
 'EX_hxa_e',
 'ARBTNR3',
 'ORNDC',
 'FA80ACPHi',
 'RNTR1c',
 'FACOAE180',
 'AGM3PApp',
 'ZN2t3pp',
 'THRt4pp',
 'AACPS5',
 'EX_gal_e',
 'CPGNR1',
 'ENTERES',
 'FUM',
 'FACOAE141',
 '2AGPGAT181',
 'AKGDH',
 'FACOAE120',
 'AACPS7',
 'AGM3PH',
 'AGM3PA',
 'PGI',
 'AGM4PCPpp',
 'GLUDy',
 '2AGPGAT120',
 'CYTBO3_4pp',
 'H2SO',
 'CDAPPA161',
 'GDPTPDP',
 'PGPP161',
 'PAPSR',
 'PTRCt2pp',
 'AACPS4',
 'LPLIPAL2E160',
 'LPLIPAL2E141',
 'TALA',
 'EX_glyc_e',
 'FACOAE161',
 'PGPP160',
 'SO4t2pp',
 'APH161',
 'RNDR4b',
 'FACOAE140',
 'MTHFC',
 'MDH',
 'PYNP2r',
 'CMPN',
 'ARBTNR2',
 'EAR181x',
 'GND',
 '4PCPpp',
 'LPLIPAL2G180',
 'FEOXAMR1',
 'PGM',
 'ZN2tpp',
 'RPE',


In [1643]:
union = pd.Series(np.union1d(w11, w12))
intersect = pd.Series(np.intersect1d(w11, w12))
bigb= union[~union.isin(intersect)]
bigb


0          AACPS2
1          ACOATA
2     ALAALAabcpp
3      ARGAGMt7pp
4           CYSDS
5        CYSabcpp
6         DKMPPD3
7          DURIPP
8         EAR100x
9         EAR121x
10        EAR141x
11         EAR40x
12         EAR60x
13         EAR80x
14       ETHAt2pp
15    EX_ala__D_e
16    EX_indole_e
18      EX_succ_e
19       EX_ura_e
20    EX_val__L_e
22          G6PDA
23           GAPD
24        GLCt2pp
25        GLYCtpp
27          IDOND
28        LALDO2x
35       MLTGY2pp
36           NTD1
37          OBTFL
38           P5CD
39            PGK
40            PGL
41          PROD2
42      SPMDabcpp
43          TMDK1
44            TPI
45          TRPS3
dtype: object

In [1644]:
i=0
intersect_decr_below_zero_and_incr_above_zero= list()
intersect_decr_below_zero_and_decr_above_zero= list()
intersect_incr_below_zero_and_decr_above_zero= list()
intersect_incr_below_zero_and_incr_above_zero= list()
intersect_decr_below_zero_and_inANDdecr_above_zero= list()
intersect_incr_below_zero_and_inANDdecr_above_zero= list()
intersect_inANDdecr_below_zero_and_decr_above_zero= list()
intersect_inANDdecr_below_zero_and_incr_above_zero= list()

while i < len(intersect):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(intersect[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True and intersect[i] not in intersect_decr_below_zero_and_decr_above_zero:
            intersect_decr_below_zero_and_decr_above_zero.append(intersect[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True and intersect[i] not in intersect_incr_below_zero_and_incr_above_zero:
            intersect_incr_below_zero_and_incr_above_zero.append(intersect[i])
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True and intersect[i] not in intersect_decr_below_zero_and_incr_above_zero:
            intersect_decr_below_zero_and_incr_above_zero.append(intersect[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True and intersect[i] not in intersect_incr_below_zero_and_decr_above_zero:
            intersect_incr_below_zero_and_decr_above_zero.append(intersect[i])
        elif my_series.is_monotonic_decreasing ==True and intersect[i] not in intersect_decr_below_zero_and_inANDdecr_above_zero:
            intersect_decr_below_zero_and_inANDdecr_above_zero.append(intersect[i])
        elif my_series.is_monotonic ==True and intersect[i] not in intersect_incr_below_zero_and_inANDdecr_above_zero:
            intersect_incr_below_zero_and_inANDdecr_above_zero.append(intersect[i])
        elif csvFile3['Biomass'].is_monotonic_decreasing ==True and intersect[i] not in intersect_inANDdecr_below_zero_and_decr_above_zero:
            intersect_inANDdecr_below_zero_and_decr_above_zero.append(intersect[i])
        elif csvFile3['Biomass'].is_monotonic ==True and intersect[i] not in intersect_inANDdecr_below_zero_and_incr_above_zero:
            intersect_inANDdecr_below_zero_and_incr_above_zero.append(intersect[i])
        
    i=i+1

intersect_decr_below_zero_and_decr_above_zero


[]

In [1645]:
intersect_incr_below_zero_and_decr_above_zero

['EX_quin_e',
 'FACOAE80',
 'GNK',
 'MDDCP1pp',
 'MDDEP3pp',
 'MLDCP1Bpp',
 'MLDCP2Bpp',
 'MLDEP2pp',
 'MLTGY1pp']

In [1646]:
i=0
bigb_decr_below_zero_and_incr_above_zero= list()
bigb_decr_below_zero_and_decr_above_zero= list()
bigb_incr_below_zero_and_decr_above_zero= list()
bigb_incr_below_zero_and_incr_above_zero= list()
bigb_decr_below_zero_and_inANDdecr_above_zero= list()
bigb_incr_below_zero_and_inANDdecr_above_zero= list()
bigb_inANDdecr_below_zero_and_decr_above_zero= list()
bigb_inANDdecr_below_zero_and_incr_above_zero= list()

while i < len(bigb):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(bigb.iloc[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True and bigb.iloc[i] not in bigb_decr_below_zero_and_decr_above_zero:
            bigb_decr_below_zero_and_decr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True and bigb.iloc[i] not in bigb_incr_below_zero_and_incr_above_zero:
            bigb_incr_below_zero_and_incr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True and bigb.iloc[i] not in bigb_decr_below_zero_and_incr_above_zero:
            bigb_decr_below_zero_and_incr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True and bigb.iloc[i] not in bigb_incr_below_zero_and_decr_above_zero:
            bigb_incr_below_zero_and_decr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic_decreasing ==True and bigb.iloc[i] not in bigb_decr_below_zero_and_inANDdecr_above_zero:
            bigb_decr_below_zero_and_inANDdecr_above_zero.append(bigb.iloc[i])
        elif my_series.is_monotonic ==True and bigb.iloc[i] not in bigb_incr_below_zero_and_inANDdecr_above_zero:
            bigb_incr_below_zero_and_inANDdecr_above_zero.append(bigb.iloc[i])
        elif csvFile3['Biomass'].is_monotonic_decreasing ==True and bigb.iloc[i] not in bigb_inANDdecr_below_zero_and_decr_above_zero:
            bigb_inANDdecr_below_zero_and_decr_above_zero.append(bigb.iloc[i])
        elif csvFile3['Biomass'].is_monotonic ==True and bigb.iloc[i] not in bigb_inANDdecr_below_zero_and_incr_above_zero:
            bigb_inANDdecr_below_zero_and_incr_above_zero.append(bigb.iloc[i])
        
    i=i+1

bigb_decr_below_zero_and_decr_above_zero


[]

In [1647]:
bigb_inANDdecr_below_zero_and_decr_above_zero

['EX_ala__D_e', 'EX_succ_e', 'GLCt2pp', 'PGK']

In [1648]:
bigb_incr_below_zero_and_inANDdecr_above_zero

['GAPD', 'PGL', 'TPI']

In [1649]:
bigb_incr_below_zero_and_decr_above_zero

['AACPS2',
 'ACOATA',
 'ALAALAabcpp',
 'ARGAGMt7pp',
 'CYSDS',
 'CYSabcpp',
 'DKMPPD3',
 'DURIPP',
 'EAR100x',
 'EAR121x',
 'EAR141x',
 'EAR40x',
 'EAR60x',
 'EAR80x',
 'ETHAt2pp',
 'EX_indole_e',
 'EX_ura_e',
 'EX_val__L_e',
 'G6PDA',
 'GLYCtpp',
 'IDOND',
 'LALDO2x',
 'MLTGY2pp',
 'NTD1',
 'OBTFL',
 'P5CD',
 'PROD2',
 'SPMDabcpp',
 'TMDK1',
 'TRPS3']

In [1650]:
bigb_inANDdecr_below_zero_and_decr_above_zero.remove("GLCt2pp")
bigb_inANDdecr_below_zero_and_decr_above_zero.remove("EX_ala__D_e")
bigb_inANDdecr_below_zero_and_decr_above_zero.remove("PGK")
bigb_incr_below_zero_and_inANDdecr_above_zero.remove("GAPD")
bigb_incr_below_zero_and_inANDdecr_above_zero.remove("PGL")
bigb_incr_below_zero_and_inANDdecr_above_zero.remove("TPI")

bigb_incr_below_zero_and_decr_above_zero.append("GLCt2pp")
decreasing_below_zero_and_decreasing_above_zero.append("EX_ala__D_e")
decreasing_below_zero_and_decreasing_above_zero.append("PGK")
bigb_incr_below_zero_and_incr_above_zero.append("GAPD")
bigb_incr_below_zero_and_decr_above_zero.append("PGL")
bigb_incr_below_zero_and_decr_above_zero.append("TPI")


In [1651]:
i=0
inANDdecr_decr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_incr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_decr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_incr_above_zero= list()

while i < len(truly_inANDdecr_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(truly_inANDdecr_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_decr_above_zero:
            inANDdecr_decr_below_zero_and_decr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_incr_above_zero:
            inANDdecr_incr_below_zero_and_incr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_incr_above_zero:
            inANDdecr_decr_below_zero_and_incr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_decr_above_zero:
            inANDdecr_incr_below_zero_and_decr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_decr_below_zero_and_inANDdecr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_incr_below_zero_and_inANDdecr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif csvFile3['Biomass'].is_monotonic_decreasing ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_decr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_decr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        elif csvFile3['Biomass'].is_monotonic ==True and truly_inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_incr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_incr_above_zero.append(truly_inANDdecr_both_positive_and_negative[i])
        
    i=i+1

inANDdecr_decr_below_zero_and_decr_above_zero


[]

In [1652]:
inANDdecr_incr_below_zero_and_decr_above_zero

['APH181',
 'PGPP180',
 'SUCASPtpp',
 'TKT1',
 'PGPP181',
 'G3PT',
 'DCTPD',
 '2AGPEAT180',
 'FECRMR2',
 'GDPMNH',
 'COBALT2t3pp',
 'CDAPPA140',
 'PGPP140',
 'RNTR4c2',
 'CYTK2',
 'CYTDH',
 'FMNRx',
 'ARBt2rpp',
 'Kt2pp',
 'MOX',
 'AACPS6',
 'FBA',
 'APH180',
 '2AGPGAT180',
 'ARBTNR3',
 'ORNDC',
 'FA80ACPHi',
 'RNTR1c',
 'FACOAE180',
 'AGM3PApp',
 'ZN2t3pp',
 'THRt4pp',
 'AACPS5',
 'CPGNR1',
 'ENTERES',
 'FACOAE141',
 '2AGPGAT181',
 'AKGDH',
 'FACOAE120',
 'AACPS7',
 'AGM3PH',
 'AGM3PA',
 'AGM4PCPpp',
 '2AGPGAT120',
 'CDAPPA161',
 'GDPTPDP',
 'PGPP161',
 'PAPSR',
 'PTRCt2pp',
 'AACPS4',
 'LPLIPAL2E160',
 'LPLIPAL2E141',
 'TALA',
 'FACOAE161',
 'PGPP160',
 'SO4t2pp',
 'APH161',
 'RNDR4b',
 'FACOAE140',
 'MTHFC',
 'PYNP2r',
 'CMPN',
 'ARBTNR2',
 'EAR181x',
 '4PCPpp',
 'LPLIPAL2G180',
 'FEOXAMR1',
 'ZN2tpp',
 'RPE',
 'LPLIPAL2G160',
 'NMNN',
 'LPLIPAL2G141',
 'RNTR3c',
 '2AGPGAT140',
 'CLPNH140pp',
 'RNTR3c2',
 'CYTDK2',
 'PGPP120',
 'ACALD',
 'ARBTNR1',
 'CLPNH141pp',
 'RNTR4c',
 'RNDR1b

In [1653]:
inANDdecr_inANDdecr_below_zero_and_decr_above_zero

['EX_ade_e',
 'EX_pro__L_e',
 'GLYCDx',
 'ME2',
 'EX_hxa_e',
 'EX_gal_e',
 'GLUDy',
 'H2SO',
 'EX_glyc_e',
 'PGM',
 'THD2pp',
 'H2Otpp',
 'CAT',
 'EX_asn__L_e',
 'PPS',
 'CO2tpp',
 'FLDR2',
 'EX_idon__L_e',
 'EX_gua_e',
 'FE3abcpp',
 'EX_cgly_e',
 'EX_etoh_e',
 'EX_alltn_e',
 'EX_12ppd__S_e',
 'EX_xan_e',
 'EX_glyc__R_e',
 'PPCK',
 'EX_4abut_e',
 'CITL',
 'EX_for_e',
 'EX_ptrc_e',
 'FBP',
 'CO2tex']

In [1654]:
inANDdecr_incr_below_zero_and_inANDdecr_above_zero

['ATPS4rpp', 'FUM', 'PGI', 'CYTBO3_4pp', 'MDH', 'GND', 'EX_co2_e', 'ENO']

In [1655]:
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("GLYCDx")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("ME2")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("GLUDy")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("THD2pp")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("FLDR2")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("PPCK")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("CITL")
inANDdecr_inANDdecr_below_zero_and_decr_above_zero.remove("FBP")
inANDdecr_incr_below_zero_and_inANDdecr_above_zero.remove("FUM")
inANDdecr_incr_below_zero_and_inANDdecr_above_zero.remove("PGI")
inANDdecr_incr_below_zero_and_inANDdecr_above_zero.remove("MDH")
inANDdecr_incr_below_zero_and_inANDdecr_above_zero.remove("GND")
inANDdecr_incr_below_zero_and_decr_above_zero.append("GLYCDx")
inANDdecr_incr_below_zero_and_decr_above_zero.append("ME2")
inANDdecr_incr_below_zero_and_decr_above_zero.append("GLUDy")
inANDdecr_incr_below_zero_and_decr_above_zero.append("THD2pp")
inANDdecr_incr_below_zero_and_decr_above_zero.append("FLDR2")
inANDdecr_incr_below_zero_and_decr_above_zero.append("PPCK")
inANDdecr_incr_below_zero_and_decr_above_zero.append("CITL")
inANDdecr_incr_below_zero_and_decr_above_zero.append("FBP")
inANDdecr_incr_below_zero_and_decr_above_zero.append("FUM")
inANDdecr_incr_below_zero_and_decr_above_zero.append("PGI")
inANDdecr_incr_below_zero_and_decr_above_zero.append("MDH")
inANDdecr_incr_below_zero_and_decr_above_zero.append("GND")


In [1656]:
len(truly_inANDdecr_both_positive_and_negative)

278

In [1657]:
len(inANDdecr_incr_below_zero_and_decr_above_zero)+len(inANDdecr_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_inANDdecr_below_zero_and_decr_above_zero)

278

In [1658]:
increasing_below_zero_and_decreasing_above_zero= list()
increasing_below_zero_and_decreasing_above_zero= list(increasing_below_zero_and_decreasing_above_zero) + list(intersect_incr_below_zero_and_decr_above_zero) + list(bigb_incr_below_zero_and_decr_above_zero) + list(inANDdecr_incr_below_zero_and_decr_above_zero)
incr_below_zero_and_inANDdecr_above_zero= list(bigb_incr_below_zero_and_inANDdecr_above_zero) + list(inANDdecr_incr_below_zero_and_inANDdecr_above_zero)


In [1659]:
inANDdecr_below_zero_and_decr_above_zero= bigb_inANDdecr_below_zero_and_decr_above_zero + inANDdecr_inANDdecr_below_zero_and_decr_above_zero


In [1660]:
increasingANDdecreasing_below_zero_and_decreasing_above_zero

['ASNN']

In [1661]:
inANDdecr_below_zero_and_decr_above_zero.append("ASNN")

In [1662]:
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
D1= pd.DataFrame (mon_incr_and_stable_only_above_zero, columns = ['Reaction names'])
D2= pd.DataFrame (mon_decr_and_stable_only_above_zero, columns = ['Reaction names'])
D4= pd.DataFrame (mon_incr_and_stable_only_below_zero, columns = ['Reaction names'])
D5= pd.DataFrame (mon_decr_and_stable_only_below_zero, columns = ['Reaction names'])
D6= pd.DataFrame (inANDdecr_and_stable_below_zero, columns = ['Reaction names'])
D7= pd.DataFrame (flat, columns = ['Reaction names'])
D8= pd.DataFrame (flat_below_zero, columns = ['Reaction names'])
D9= pd.DataFrame (flat_both_positive_and_negative, columns = ['Reaction names'])
D10= pd.DataFrame (increasing_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D11= pd.DataFrame (increasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D12= pd.DataFrame (flat_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D13= pd.DataFrame (decreasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D15= pd.DataFrame (increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D16= pd.DataFrame (inANDdecr_below_zero_and_decr_above_zero, columns = ['Reaction names'])
D17= pd.DataFrame (increasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D18= pd.DataFrame (increasing_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D19= pd.DataFrame (flat_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D20= pd.DataFrame (decr_above_zero, columns = ['Reaction names'])
D21= pd.DataFrame (one_to_three_data_point, columns = ['Reaction names'])
D22= pd.DataFrame (decr_below_zero, columns = ['Reaction names'])
D23= pd.DataFrame (incr_below_zero, columns = ['Reaction names'])
D24= pd.DataFrame (truly_inANDdecr_below_zero, columns = ['Reaction names'])
D25= pd.DataFrame (decreasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D26= pd.DataFrame (increasing_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
D27= pd.DataFrame (incr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D28= pd.DataFrame (increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D29= pd.DataFrame (flat_below_zero_and_increasingANDdecreasing_above_zero, columns = ['Reaction names'])
D30= pd.DataFrame (flat_below_zero_and_increasing_above_zero, columns = ['Reaction names'])


In [1663]:
len(D)+len(D1)+len(D2)+len(D4)+len(D5)+len(D6)+len(D7)+len(D8)+len(D9)+len(D10)+len(D11)+len(D12)+len(D13)+len(D15)+len(D16)+len(D17)+len(D18)+len(D19)+len(D20)+len(D21)+len(D22)+len(D23)+len(D24)+len(D25)+len(D26)+len(D27)+len(D28)+len(D29)+len(D30)


2605

In [1664]:
i=0
one_to_three_data_point_below_zero= list()
one_to_three_data_point_above_zero= list()

while i < len(flat_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(flat_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero.append(flat_both_positive_and_negative[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero.append(flat_both_positive_and_negative[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero


['ACACT1r', 'ACACT2r', 'ACACT3r', 'HACD1', 'HACD2', 'HACD3', 'INDOLEt2rpp']

In [1665]:
one_to_three_data_point_above_zero

['EAR40y',
 'EAR60y',
 'EAR80y',
 'ICHORS_copy1',
 'KAS14',
 'KAS15',
 'MDDCP3pp',
 'MLDCP2App',
 'TRPS1']

In [1666]:
i=0
F_below_zero_and_IorD_above_zero= list()
F_below_zero_and_ISorDS_above_zero= list()
IorD_below_zero_and_F_above_zero= list()
ISorDS_below_zero_and_F_above_zero= list()
F_below_zero_and_F_above_zero= list()
r_id ="ACACT8r_copy1"
csvFile = pd.read_csv('biomass_per_flux_for_reaction_ACACT8r_copy1_lactose.csv')
l= list()

while i < 101:
    inf= csvFile.iloc[i,  2]
    flt= float(csvFile.iloc[i,  2])
    
    if csvFile.iloc[i,  2] == flt:
        l.append(csvFile.iloc[i,  2])
    i= i+1

csvFile2= pd.DataFrame (l, columns = ['Biomass'])
csvFile2.Biomass = csvFile2.Biomass.round(2)

my_series = csvFile2['Biomass'].squeeze()

if len(csvFile2['Biomass'])>3:
        
    if len(my_series.unique())>3 and len(my_series.unique())>len(my_series)-4 and r_id not in ISorDS_below_zero_and_F_above_zero:
        IorD_below_zero_and_F_above_zero.append(r_id)
    elif len(my_series.unique())>3 and len(my_series.unique())<len(my_series)-4 and r_id not in IorD_below_zero_and_F_above_zero:
        ISorDS_below_zero_and_F_above_zero.append(r_id)
    
l= list()
i=100
        
while i < 201:
    inf= csvFile.iloc[i,  2]
    flt= float(csvFile.iloc[i,  2])
    
    if csvFile.iloc[i,  2] == flt:
        l.append(csvFile.iloc[i,  2])
    i= i+1

csvFile3= pd.DataFrame (l, columns = ['Biomass'])
csvFile3.Biomass = csvFile3.Biomass.round(2)

my_series2 = csvFile3['Biomass'].squeeze()

if len(csvFile3['Biomass'])>3:
        
    if len(my_series2.unique())>3 and len(my_series2.unique())>len(my_series2)-4 and r_id not in F_below_zero_and_ISorDS_above_zero:
        F_below_zero_and_IorD_above_zero.append(r_id)
    elif len(my_series2.unique())>3 and len(my_series2.unique())<len(my_series2)-4 and r_id not in F_below_zero_and_IorD_above_zero:
        F_below_zero_and_ISorDS_above_zero.append(r_id)
    else:
        if r_id not in (ISorDS_below_zero_and_F_above_zero or IorD_below_zero_and_F_above_zero):
            F_below_zero_and_F_above_zero.append(r_id)

F_below_zero_and_F_above_zero


[]

In [1667]:
F_below_zero_and_IorD_above_zero

['ACACT8r_copy1']

In [1668]:
set1= set(flat_both_positive_and_negative)
set2= set(one_to_three_data_point_below_zero)
set3= set(one_to_three_data_point_above_zero)
truly_flat_both_positive_and_negative= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_flat_above_zero= list(one_to_three_data_point_below_zero)
flat_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero)

i3= 0
F_below_zero_and_IorD_above_zero= list()
F_below_zero_and_ISorDS_above_zero= list()
IorD_below_zero_and_F_above_zero= list()
ISorDS_below_zero_and_F_above_zero= list()
F_below_zero_and_F_above_zero= list()
r_id = truly_flat_both_positive_and_negative[0]

for r_id in truly_flat_both_positive_and_negative:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = truly_flat_both_positive_and_negative[i3]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>3:
        
        if len(my_series.unique())>3 and len(my_series.unique())>len(my_series)-4 and r_id not in ISorDS_below_zero_and_F_above_zero:
            IorD_below_zero_and_F_above_zero.append(r_id)
        elif len(my_series.unique())>3 and len(my_series.unique())<len(my_series)-4 and r_id not in IorD_below_zero_and_F_above_zero:
            ISorDS_below_zero_and_F_above_zero.append(r_id)
    
    l= list()
    i=100
        
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile3= pd.DataFrame (l, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)

    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile3['Biomass'])>3:
        
        if len(my_series2.unique())>3 and len(my_series2.unique())>len(my_series2)-4 and r_id not in F_below_zero_and_ISorDS_above_zero:
            F_below_zero_and_IorD_above_zero.append(r_id)
        elif len(my_series2.unique())>3 and len(my_series2.unique())<len(my_series2)-4 and r_id not in F_below_zero_and_IorD_above_zero:
            F_below_zero_and_ISorDS_above_zero.append(r_id)
        else:
            if r_id not in (ISorDS_below_zero_and_F_above_zero or IorD_below_zero_and_F_above_zero):
                F_below_zero_and_F_above_zero.append(r_id)

    
    i3= i3 + 1

F_below_zero_and_F_above_zero = [*set(F_below_zero_and_F_above_zero)]
F_below_zero_and_F_above_zero


['ACACT9',
 'GLCtex_copy2',
 'CTECOAI6_copy2',
 'KAT1',
 'VPAMTr_copy1',
 'ACACT8',
 'URAt2pp_copy1',
 'SERt4pp',
 'ASPt2pp_copy1',
 'ACACT11',
 'CRNt8pp',
 'HACD8i',
 'GLDBRAN2',
 'HACD1i',
 'GLCP2',
 'ALAt4pp',
 'DHNPA2r_copy2',
 'ACACT10',
 'ALAt2pp_copy1',
 'VPAMTr_copy2',
 'DARBt2rpp_copy2',
 'VALTA',
 'PROt4pp',
 'GLUt4pp',
 'ACS',
 'ICHORS_copy2',
 'NDPK1',
 'ADK1',
 'MICITDr_copy2',
 'CTECOAI7_copy2',
 'HACD7i',
 'ACACT13',
 'GLYCLTt4pp',
 'CA2t3pp',
 'ACACT8r_copy2',
 'HACD2i',
 'GLYt2pp_copy1',
 'INDOLEt2pp',
 'ALDD19xr_copy2',
 'L_LACtex_copy1',
 'ADK3',
 'GLCP',
 'HACD5i',
 'ABUTt2pp',
 'L_LACtex_copy2',
 'HACD6i',
 'ACACT12',
 'NAt3pp',
 'HACD4i',
 'HACD3i',
 'ACt4pp',
 'CTECOAI8_copy2',
 'HPYRRx',
 'GLYt4pp',
 'ALATA_L']

In [1669]:
I_below_zero_and_F_above_zero= list(IorD_below_zero_and_F_above_zero)
increasing_below_zero_and_flat_above_zero= list(increasing_below_zero_and_flat_above_zero) + list(I_below_zero_and_F_above_zero)

F_below_zero_and_D_above_zero= list(F_below_zero_and_IorD_above_zero)
flat_below_zero_and_decreasing_above_zero= list(flat_below_zero_and_decreasing_above_zero) + list(F_below_zero_and_D_above_zero)


In [1670]:
i=0
one_to_three_data_point_below_zero2= list()
one_to_three_data_point_above_zero2= list()

while i < len(increasing_and_stable_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(increasing_and_stable_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero2.append(increasing_and_stable_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero2.append(increasing_and_stable_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero2


[]

In [1671]:
one_to_three_data_point_above_zero2

['EX_lys__L_e']

In [1672]:
i=0
one_to_three_data_point_below_zero3= list()
one_to_three_data_point_above_zero3= list()

while i < len(decreasing_and_stable_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(decreasing_and_stable_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero3.append(decreasing_and_stable_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero3.append(decreasing_and_stable_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero3


[]

In [1673]:
one_to_three_data_point_above_zero3

['EX_glcn_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'EX_alaala_e',
 'EX_arg__L_e',
 'EX_orn_e',
 'GLCabcpp',
 'GALabcpp']

In [1674]:
i=0
one_to_three_data_point_below_zero4= list()
one_to_three_data_point_above_zero4= list()

while i < len(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero4.append(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero4.append(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero4


[]

In [1675]:
one_to_three_data_point_above_zero4

['EX_agm_e', 'EX_glc__D_e']

In [1676]:
i=0
one_to_three_data_point_below_zero5= list()
one_to_three_data_point_above_zero5= list()

while i < len(inANDdecr_below_zero_and_decr_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(inANDdecr_below_zero_and_decr_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero5.append(inANDdecr_below_zero_and_decr_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero5.append(inANDdecr_below_zero_and_decr_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero5


[]

In [1677]:
one_to_three_data_point_above_zero5

['EX_ade_e',
 'EX_pro__L_e',
 'EX_hxa_e',
 'EX_gal_e',
 'EX_idon__L_e',
 'EX_gua_e',
 'EX_cgly_e',
 'EX_12ppd__S_e',
 'EX_xan_e',
 'EX_4abut_e',
 'EX_ptrc_e']

In [1678]:
i=0
one_to_three_data_point_below_zero6= list()
one_to_three_data_point_above_zero6= list()

while i < len(increasing_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(increasing_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero6.append(increasing_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero6.append(increasing_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero6


['GNK',
 'MDDCP1pp',
 'MDDEP3pp',
 'MLDCP1Bpp',
 'MLDCP2Bpp',
 'MLDEP2pp',
 'ALAALAabcpp',
 'CYSDS',
 'CYSabcpp',
 'DURIPP',
 'G6PDA',
 'NTD1',
 'OBTFL',
 'P5CD',
 'PROD2',
 'SPMDabcpp',
 'TMDK1',
 'PGL']

In [1679]:
one_to_three_data_point_above_zero6

['EX_quin_e',
 'GNK',
 'MDDCP1pp',
 'MDDEP3pp',
 'MLDCP1Bpp',
 'MLDCP2Bpp',
 'MLDEP2pp',
 'MLTGY1pp',
 'ACOATA',
 'ARGAGMt7pp',
 'DKMPPD3',
 'EAR40x',
 'EAR60x',
 'EAR80x',
 'EX_indole_e',
 'EX_val__L_e',
 'IDOND',
 'LALDO2x',
 'MLTGY2pp',
 'TRPS3',
 'GLCt2pp',
 'TPI']

In [1680]:
i=0
one_to_three_data_point_below_zero7= list()
one_to_three_data_point_above_zero7= list()

while i < len(decreasing_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(decreasing_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero7.append(decreasing_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero7.append(decreasing_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero7


[]

In [1681]:
one_to_three_data_point_above_zero7

['EX_glu__L_e',
 'EX_glyc3p_e',
 'EX_5dglcn_e',
 'EX_akg_e',
 'EX_etha_e',
 'EX_thr__L_e',
 'EX_g3pe_e',
 'EX_g3pg_e',
 'EX_ala__D_e']

In [1682]:
i=0
one_to_three_data_point_below_zero8= list()
one_to_three_data_point_above_zero8= list()

while i < len(increasing_below_zero_and_increasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(increasing_below_zero_and_increasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero:
        
        one_to_three_data_point_below_zero8.append(increasing_below_zero_and_increasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero:
        
        one_to_three_data_point_above_zero8.append(increasing_below_zero_and_increasing_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero8


[]

In [1683]:
one_to_three_data_point_above_zero8

['GLCptspp', 'GALt2pp', 'LACZ', 'PGMT']

In [1684]:
i=0
one_to_three_data_point3_below_zero= list()
one_to_three_data_point3_above_zero= list()

while i < len(one_to_three_data_point3):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(one_to_three_data_point3[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point3_below_zero:
        
        one_to_three_data_point3_below_zero.append(one_to_three_data_point3[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point3_above_zero:
        
        one_to_three_data_point3_above_zero.append(one_to_three_data_point3[i])
    
    i= i + 1
    
one_to_three_data_point3_below_zero


['ACOAD1f', 'ACOAD2f', 'ECOAH1', 'ECOAH2']

In [1685]:
one_to_three_data_point3_above_zero


['ACOAD1f', 'ACOAD2f', 'ECOAH1', 'ECOAH2']

In [1686]:
i=0
one_to_three_data_point_below_zero10= list()
one_to_three_data_point_above_zero10= list()

while i < len(flat):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(flat[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero10:
        
        one_to_three_data_point_below_zero10.append(flat[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero10:
        
        one_to_three_data_point_above_zero10.append(flat[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero10


[]

In [1687]:
one_to_three_data_point_above_zero10

['LACZpp']

In [1688]:
i=0
one_to_three_data_point_below_zero11= list()
one_to_three_data_point_above_zero11= list()

while i < len(flat_below_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(flat_below_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2=100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero11:
        
        one_to_three_data_point_below_zero11.append(flat_below_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero11:
        
        one_to_three_data_point_above_zero11.append(flat_below_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero11

[]

In [1689]:
one_to_three_data_point_above_zero11

[]

In [1690]:
set1= set(one_to_three_data_point3)
set2= set(one_to_three_data_point3_below_zero)
set3= set(one_to_three_data_point3_above_zero)
truly_one_to_three_data_point3= list(set1-set2-set3)
truly_one_to_three_data_point3


[]

In [1691]:
set1= set(increasing_and_stable_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero2)
set3= set(one_to_three_data_point_above_zero2)
truly_increasing_and_stable_below_zero_and_decreasing_above_zero= list(set1-set2-set3)

set1= set(decreasing_and_stable_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero3)
set3= set(one_to_three_data_point_above_zero3)
truly_decreasing_and_stable_below_zero_and_decreasing_above_zero= list(set1-set2-set3)

set1= set(increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero4)
set3= set(one_to_three_data_point_above_zero4)
truly_increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero= list(set1-set2-set3)

set1= set(inANDdecr_below_zero_and_decr_above_zero)
set2= set(one_to_three_data_point_below_zero5)
set3= set(one_to_three_data_point_above_zero5)
truly_inANDdecr_below_zero_and_decr_above_zero= list(set1-set2-set3)

set1= set(increasing_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero6)
set3= set(one_to_three_data_point_above_zero6)
truly_increasing_below_zero_and_decreasing_above_zero= list(set1-set2-set3)
truly_increasing_below_zero_and_decreasing_above_zero.append("NTD6")

set1= set(decreasing_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_below_zero7)
set3= set(one_to_three_data_point_above_zero7)
truly_decreasing_below_zero_and_decreasing_above_zero= list(set1-set2-set3)

set1= set(increasing_below_zero_and_increasing_above_zero)
set2= set(one_to_three_data_point_below_zero8)
set3= set(one_to_three_data_point_above_zero8)
truly_increasing_below_zero_and_increasing_above_zero= list(set1-set2-set3)
truly_increasing_below_zero_and_increasing_above_zero.append("GAPD")

set1= set(one_to_three_data_point3)
set2= set(one_to_three_data_point3_below_zero)
set3= set(one_to_three_data_point3_above_zero)
truly_one_to_three_data_point3= list(set1-set2-set3)

set1= set(flat)
set2= set(one_to_three_data_point_below_zero10)
set3= set(one_to_three_data_point_above_zero10)
truly_flat_above_zero= list(set1-set2-set3)

set1= set(flat_below_zero)
set2= set(one_to_three_data_point_below_zero11)
set3= set(one_to_three_data_point_above_zero11)
truly_flat_below_zero= list(set1-set2-set3)

intersectI = pd.Series(np.intersect1d(one_to_three_data_point_below_zero, one_to_three_data_point_above_zero))
intersectII = pd.Series(np.intersect1d(one_to_three_data_point_below_zero2, one_to_three_data_point_above_zero2))
intersectIII = pd.Series(np.intersect1d(one_to_three_data_point_below_zero3, one_to_three_data_point_above_zero4))
intersectIV = pd.Series(np.intersect1d(one_to_three_data_point_below_zero4, one_to_three_data_point_above_zero4))
intersectV = pd.Series(np.intersect1d(one_to_three_data_point_below_zero5, one_to_three_data_point_above_zero5))
intersectVI = pd.Series(np.intersect1d(one_to_three_data_point_below_zero6, one_to_three_data_point_above_zero6))
intersectVII = pd.Series(np.intersect1d(one_to_three_data_point_below_zero7, one_to_three_data_point_above_zero7))
intersectVIII = pd.Series(np.intersect1d(one_to_three_data_point_below_zero8, one_to_three_data_point_above_zero8))
intersectX = pd.Series(np.intersect1d(one_to_three_data_point3_below_zero, one_to_three_data_point3_above_zero))

truly_one_to_three_data_point_both_positive_and_negative= list(intersectVI)+list(intersectX)

set1= set(one_to_three_data_point_below_zero6)
set2= set(intersectVI)
one_to_three_data_point_below_zero6= list(set1-set2)

set3= set(one_to_three_data_point_above_zero6)
set4= set(intersectVI)
one_to_three_data_point_above_zero6= list(set3-set4)

truly_one_to_three_data_point_below_zero= list(one_to_three_data_point2)+list(one_to_three_data_point_below_zero11)
truly_one_to_three_data_point_above_zero= list(one_to_three_data_point)+list(one_to_three_data_point_above_zero10)


one_to_three_data_point_below_zero_and_decreasing_above_zero= list(one_to_three_data_point_below_zero2)+list(one_to_three_data_point_below_zero3)+list(one_to_three_data_point_below_zero4)+list(one_to_three_data_point_below_zero5)+list(one_to_three_data_point_below_zero6)+list(one_to_three_data_point_below_zero7)
increasing_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero2)
decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero3)
increasingANDdecreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero4)

inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero5)
increasing_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero6)+list(one_to_three_data_point_above_zero8)
decreasing_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero7)

one_to_three_data_point_below_zero_and_increasing_above_zero= list(one_to_three_data_point_below_zero8)


In [1692]:
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
D1= pd.DataFrame (mon_incr_and_stable_only_above_zero, columns = ['Reaction names'])
D2= pd.DataFrame (mon_decr_and_stable_only_above_zero, columns = ['Reaction names'])
D3= pd.DataFrame (one, columns = ['Reaction names'])
D4= pd.DataFrame (mon_incr_and_stable_only_below_zero, columns = ['Reaction names'])
D5= pd.DataFrame (mon_decr_and_stable_only_below_zero, columns = ['Reaction names'])
D6= pd.DataFrame (inANDdecr_and_stable_below_zero, columns = ['Reaction names'])
D7= pd.DataFrame (truly_flat_above_zero, columns = ['Reaction names'])
D8= pd.DataFrame (truly_flat_below_zero, columns = ['Reaction names'])
D9= pd.DataFrame (F_below_zero_and_F_above_zero, columns = ['Reaction names'])
D10= pd.DataFrame (increasing_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D11= pd.DataFrame (truly_increasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D12= pd.DataFrame (flat_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D13= pd.DataFrame (truly_decreasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D14= pd.DataFrame (truly_inANDdecr_above_zero, columns = ['Reaction names'])
D15= pd.DataFrame (truly_increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D16= pd.DataFrame (truly_inANDdecr_below_zero_and_decr_above_zero, columns = ['Reaction names'])
D17= pd.DataFrame (truly_increasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D18= pd.DataFrame (increasing_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D19= pd.DataFrame (flat_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D20= pd.DataFrame (decr_above_zero, columns = ['Reaction names'])
D22= pd.DataFrame (decr_below_zero, columns = ['Reaction names'])
D23= pd.DataFrame (incr_below_zero, columns = ['Reaction names'])
D24= pd.DataFrame (truly_inANDdecr_below_zero, columns = ['Reaction names'])
D25= pd.DataFrame (truly_decreasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D26= pd.DataFrame (truly_increasing_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
D27= pd.DataFrame (truly_incr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D28= pd.DataFrame (increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D29= pd.DataFrame (truly_one_to_three_data_point_below_zero, columns = ['Reaction names'])
D30= pd.DataFrame (truly_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D31= pd.DataFrame (truly_one_to_three_data_point_both_positive_and_negative, columns = ['Reaction names'])
D32= pd.DataFrame (one_to_three_data_point_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D33= pd.DataFrame (flat_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D34= pd.DataFrame (one_to_three_data_point_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D35= pd.DataFrame (increasing_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D36= pd.DataFrame (decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D37= pd.DataFrame (increasingANDdecreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D38= pd.DataFrame (inANDdecr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D39= pd.DataFrame (increasing_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D40= pd.DataFrame (decreasing_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D41= pd.DataFrame (one_to_three_data_point_below_zero_and_increasing_above_zero, columns = ['Reaction names'])
D42= pd.DataFrame (increasing_below_zero_and_increasingANDdecreasing_and_stable_above_zero, columns = ['Reaction names'])
D43= pd.DataFrame (inANDdecr_and_stable, columns = ['Reaction names'])


In [1693]:
len(D)+len(D1)+len(D2)+len(D3)+len(D4)+len(D5)+len(D6)+len(D7)+len(D8)+len(D9)+len(D10)+len(D11)+len(D12)+len(D13)+len(D14)+len(D15)+len(D16)+len(D17)+len(D18)+len(D19)+len(D20)+len(D22)+len(D23)+len(D24)+len(D25)+len(D26)+len(D27)+len(D28)+len(D29)+len(D30)+len(D31)+len(D32)+len(D33)+len(D34)+len(D35)+len(D36)+len(D37)+len(D38)+len(D39)+len(D40)+len(D41)+len(D42)+len(D43)


2726

In [1694]:
D.to_csv("NEW one feasible value at zero reactions lactose.csv")
D1.to_csv("NEW increasing and stable above zero lactose.csv")
D2.to_csv("NEW decreasing and stable above zero lactose.csv")
D3.to_csv("NEW one feasible value NOT at zero reactions lactose.csv")
D4.to_csv("NEW decreasing and stable below zero lactose.csv")
D5.to_csv("NEW increasing and stable below zero lactose.csv")
D6.to_csv("NEW increasing and decreasing and stable below zero lactose.csv")
D7.to_csv("NEW flat above zero lactose.csv")
D8.to_csv("NEW flat below zero lactose.csv")
D9.to_csv("NEW flat both positive and negative lactose.csv")
D10.to_csv("NEW decreasing below zero and flat above zero lactose.csv")
D11.to_csv("NEW decreasing and stable below zero and decreasing above zero lactose.csv")
D12.to_csv("NEW flat below zero and decreasing above zero lactose.csv")
D13.to_csv("NEW increasing and stable only below zero and decreasing above zero lactose.csv")
D14.to_csv("NEW increasing and decreasing above zero lactose.csv")
D15.to_csv("NEW increasing and decreasing and stable below zero and decreasing above zero lactose.csv")
D16.to_csv("NEW increasing and decreasing below zero and decreasing above zero lactose.csv")
D17.to_csv("NEW decreasing below zero and decreasing above zero lactose.csv")
D18.to_csv("NEW decreasing below zero and decreasing and stable above zero lactose.csv")
D19.to_csv("NEW flat_below_zero_and_decreasing_and_stable_above_zero lactose.csv")
D20.to_csv("NEW decreasing above zero lactose.csv")
D22.to_csv("NEW increasing below zero lactose.csv")
D23.to_csv("NEW decreasing below zero lactose.csv")
D24.to_csv("NEW increasing and decreasing below zero lactose.csv")
D25.to_csv("NEW increasing below zero and decreasing above zero lactose.csv")
D26.to_csv("NEW decreasing below zero and increasing above zero lactose.csv")
D27.to_csv("NEW decreasing below zero and increasing and decreasing above zero lactose.csv")
D28.to_csv("NEW decreasing and stable below zero and decreasing and stable above zero lactose.csv")
D29.to_csv("NEW truly_one_to_three_data_point_below_zero lactose.csv")
D30.to_csv("NEW truly_one_to_three_data_point_above_zero lactose.csv")
D31.to_csv("NEW truly_one_to_three_data_point_both_positive_and_negative lactose.csv")
D32.to_csv("NEW one_to_three_data_point_below_zero_and_flat_above_zero lactose.csv")
D33.to_csv("NEW flat_below_zero_and_one_to_three_data_point_above_zero lactose.csv")
D34.to_csv("NEW one_to_three_data_point_below_zero_and_decreasing_above_zero lactose.csv")
D35.to_csv("NEW decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero lactose.csv")
D36.to_csv("NEW increasing_and_stable_below_zero_and_one_to_three_data_point_above_zero lactose.csv")
D37.to_csv("NEW increasingANDdecreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero lactose.csv")
D38.to_csv("NEW inANDdecr_below_zero_and_one_to_three_data_point_above_zero lactose.csv")
D39.to_csv("NEW decreasing_below_zero_and_one_to_three_data_point_above_zero lactose.csv")
D40.to_csv("NEW increasing_below_zero_and_one_to_three_data_point_above_zero lactose.csv")
D41.to_csv("NEW one_to_three_data_point_below_zero_and_increasing_above_zero lactose.csv")
D42.to_csv("NEW decreasing_below_zero_and_increasingANDdecreasing_and_stable_above_zero")
D43.to_csv("NEW increasing and decreasing and stable above zero lactose")


In [1695]:
Dtotal= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_only_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_only_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one))
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_only_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_only_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_flat_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(F_below_zero_and_F_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_increasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_decreasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_inANDdecr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_increasingANDdecreasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_inANDdecr_below_zero_and_decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_increasing_below_zero_and_decreasing_above_zero))   
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(incr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_inANDdecr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_decreasing_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_increasing_below_zero_and_increasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_incr_below_zero_and_inANDdecr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_one_to_three_data_point_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(truly_one_to_three_data_point_both_positive_and_negative))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasingANDdecreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_below_zero_and_one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_increasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_increasingANDdecreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable))
              
Dtotal.to_csv("all reactions lactose.csv")
